In [5]:
import numpy as np
import random
from random import randrange
from queue import PriorityQueue
import math
import sys
import torch
import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import matplotlib.pyplot as plt
import numpy as np
import torch.nn as nn
import torch.optim as optim
import random
np.set_printoptions(threshold=sys.maxsize)

In [6]:
#Michael McMahon mam1198

def bfs(ship, botrow, botcol, d, mouserow, mousecol):
    fringe = [(botrow, botcol)]
    #parent: dictionary: keys: child. values: parent
    parent = {}
    dist = np.ones((d, d)) * np.inf
    dist[botrow, botcol] = 0
    path = []
    while len(fringe) != 0:
        neighbors = []
        curr = fringe.pop(0)
        currrow = curr[0]
        currcol = curr[1]
        #check up
        if currrow > 0:
            if ship[currrow-1, currcol] != 1:
                if dist[currrow-1, currcol] == np.inf:
                    dist[currrow-1, currcol] = dist[currrow, currcol] + 1
                    parent[(currrow-1, currcol)] = (currrow, currcol)
                    fringe.append((currrow-1, currcol))
        #check down
        if currrow < d-1:
            if ship[currrow+1, currcol] != 1:
                if dist[currrow+1, currcol] == np.inf:
                    dist[currrow+1, currcol] = dist[currrow, currcol] + 1
                    parent[(currrow+1, currcol)] = (currrow, currcol)
                    fringe.append((currrow+1, currcol))
        #check left
        if currcol > 0:
            if ship[currrow, currcol-1] != 1:
                if dist[currrow, currcol-1] == np.inf:
                    dist[currrow, currcol-1] = dist[currrow, currcol] + 1
                    parent[(currrow, currcol-1)] = (currrow, currcol)
                    fringe.append((currrow, currcol-1))
        #check right
        if currcol < d-1:
            if ship[currrow, currcol+1] != 1:
                if dist[currrow, currcol+1] == np.inf:
                    dist[currrow, currcol+1] = dist[currrow, currcol] + 1
                    parent[(currrow, currcol+1)] = (currrow, currcol)
                    fringe.append((currrow, currcol+1))
    #if (mouserow, mousecol) in parent:
    current = (mouserow, mousecol)
    path.append(current)
    while(1):
        if current in parent:
            current = parent[current]
            path.append(current)
        else:
            break
    path.reverse()
    return path

#Returns the number of adjacent open cells for a block
def check_adj(i, j, ship, d):
    k = 0
    #check up
    if i > 0:
        if ship[i-1, j] != 1:
            k += 1
    #check down
    if i < d-1:
        if ship[i+1, j] != 1:
            k += 1
    #check left
    if j > 0:
        if ship[i, j-1] != 1:
            k +=1
    #check right
    if j < d-1:
        if ship[i, j+1] != 1:
            k +=1
    return k

#Build the ship
#d: dimensions, num_mice: number of mice
def construct(d, num_mice):
    ship = np.ones((d, d))
    open_row = randrange(d)
    open_col = randrange(d)
    ship[open_row, open_col] = 0
    c = 0
    a=0
    while(c==0):
        blocked1n = []
        blocked1n_orig = []
        #Identify all currently blocked cells that have exactly one neighbor
        for i in range(d):
            for j in range(d):
                open_neighbors = 0
                curr = ship[i, j]
                if curr == 1:
                    #check up
                    if i > 0:
                        if ship[i-1, j] == 0:
                            open_neighbors += 1
                    #check down
                    if i < d-1:
                        if ship[i+1, j] == 0:
                            open_neighbors += 1
                    #check left
                    if j > 0:
                        if ship[i, j-1] == 0:
                            open_neighbors +=1
                    #check right
                    if j < d-1:
                        if ship[i, j+1] == 0:
                            open_neighbors +=1
                    if open_neighbors == 1:
                        blocked1n.append((i, j))
        blocked1n_orig = blocked1n.copy()
        #blocked1n: list of currently blocked cells that have exactly one open neighbor
        #Pick one at random and open the selected cell
        if len(blocked1n) > 0:
            ropen = randrange(len(blocked1n))
            rcell = blocked1n[ropen]
            ship[rcell[0], rcell[1]] = 0
        if (len(blocked1n_orig)) == 0:
            c+=1
    #Identify dead ends: open cells with one open neighbor
    #open1n: list of open cells with exactly one open neighbor
    open1n = []
    for i in range(d):
        for j in range(d):
            open_neighbors = 0
            curr = ship[i, j]
            if curr == 0:
                #check up
                if i > 0:
                    if ship[i-1, j] == 0:
                        open_neighbors += 1
                #check down
                if i < d-1:
                    if ship[i+1, j] == 0:
                        open_neighbors += 1
                #check left
                if j > 0:
                    if ship[i, j-1] == 0:
                        open_neighbors +=1
                #check right
                if j < d-1:
                    if ship[i, j+1] == 0:
                        open_neighbors +=1
                if open_neighbors == 1:
                    open1n.append((i, j))
    #For approximately half of these cells, pick one of their closed neighbors at random and open it
    #Closed cell = 1, open cell = 0
    for i in range(int(len(open1n)/2)):
        next_ind = randrange(len(open1n))
        d_end = open1n[next_ind]
        dir_to_open = randrange(4)
        found = 0
        for i in range(4):
            #check up
            if dir_to_open == 0:
                if d_end[0] > 0:
                    if ship[d_end[0]-1, d_end[1]] == 1:
                        ship[d_end[0]-1, d_end[1]] = 0
                        break
            #check down
            if dir_to_open == 1:
                if d_end[0] < d-1:
                    if ship[d_end[0]+1, d_end[1]] == 1:
                        ship[d_end[0]+1, d_end[1]] = 0
                        break
            #check left
            if dir_to_open == 2:
                if d_end[1] > 0:
                    if ship[d_end[0], d_end[1]-1] == 1:
                        ship[d_end[0], d_end[1]-1] = 0
                        break
            #check right
            if dir_to_open == 3:
                if d_end[1] < d-1:
                    if ship[d_end[0], d_end[1]+1] == 1:
                        ship[d_end[0], d_end[1]+1] = 0
                        break
            dir_to_open = randrange(4)
        open1n.remove(d_end)
    if len(open1n) == 1:
        d_end = open1n[0]
        while(found==0):
            #check up
            if dir_to_open == 0:
                if d_end[0] > 0:
                    if ship[d_end[0]-1, d_end[1]] == 1:
                        ship[d_end[0]-1, d_end[1]] = 0
                        found += 1
            #check down
            if dir_to_open == 1:
                if d_end[0] < d-1:
                    if ship[d_end[0]+1, d_end[1]] == 1:
                        ship[d_end[0]+1, d_end[1]] = 0
                        found += 1
            #check left
            if dir_to_open == 2:
                if d_end[1] > 0:
                    if ship[d_end[0], d_end[1]-1] == 1:
                        ship[d_end[0], d_end[1]-1] = 0
                        found += 1
            #check right
            if dir_to_open == 3:
                if d_end[1] < d-1:
                    if ship[d_end[0], d_end[1]+1] == 1:
                        ship[d_end[0], d_end[1]+1] = 0
                        found += 1
            dir_to_open = randrange(4)
        open1n.remove(open1n[0])
    botrow = randrange(d)
    botcol = randrange(d)
    #Bot = 2
    while ship[botrow, botcol] != 0:
        botrow = randrange(d)
        botcol = randrange(d)
    ship[botrow, botcol] = 2
    mouserow = randrange(d)
    mousecol = randrange(d)
    #Mouse = 3
    while ship[mouserow, mousecol] != 0:
        mouserow = randrange(d)
        mousecol = randrange(d)
    ship[mouserow, mousecol] = 3
    if num_mice == 2:
        mouserow2 = randrange(d)
        mousecol2 = randrange(d)
        double = 0
        if ship[mouserow2, mousecol2] == 3:
            #Found a mouse, 4 will represent both mice occupying the same cell
            double = 1
        while ship[mouserow2, mousecol2] != 0:
            mouserow2 = randrange(d)
            mousecol2 = randrange(d)
            if ship[mouserow2, mousecol2] == 3:
                #Found a mouse, 4 will represent both mice occupying the same cell
                double = 1
        if double == 0:
            ship[mouserow2, mousecol2] = 3
        else:
            ship[mouserow2, mousecol2] = 4
    return ship, botrow, botcol, mouserow, mousecol

#Initializes the probability grid for a given ship
def probbuild(ship):
    pcount = 0
    pindices = []
    probgrid = np.zeros((len(ship[0]), len(ship[0])))
    for i in range(len(ship[0])):
        for j in range(len(ship[0])):
            if ship[i, j] == 0 or ship[i, j] == 3:
                pcount += 1
                pindices.append((i, j))
    for i in range(pcount):
        probgrid[pindices[i][0], pindices[i][1]] = 1 / pcount
    return probgrid

#Updates the probabilities in the grid in the case of a beep from P(A) to P(A|B)
#Beep: 1 if there was a beep, 0 if not
#P(A|B) =  P(A)*e^(−α∗(d−1)) / Σ(i=1, pcount) P(Ai)*e^(−α∗(d−1))
#Let Σ(i=1, pcount) P(Ai)*e^(−α∗(d−1)) = Pb
def response(probgrid, a, botrow, botcol, beep):
    newgrid = np.zeros((len(probgrid[0]), len(probgrid[0])))
    Pb = 0
    #Sum up the products of the possibilities and e^(−α∗(d−1)) for each possible cell as P(B)
    for i in range(len(probgrid[0])):
        for j in range(len(probgrid[0])):
            if probgrid[i, j] != 0:
                d = abs(i - botrow) + abs(j - botcol)
                if beep == 1:
                    bprob = math.e ** (a*(-1)*(d-1))
                else:
                    bprob = 1 - (math.e ** (a*(-1)*(d-1)))
                Pb += (probgrid[i, j] * bprob)
                newgrid[i, j] = probgrid[i, j] * bprob
    #Update the probabilities of each cell to P(A|B)
    for i in range(len(probgrid[0])):
        for j in range(len(probgrid[0])):
            if probgrid[i, j] != 0:
                d = abs(i - botrow) + abs(j - botcol)
                if beep == 1:
                    bprob = math.e ** (a*(-1)*(d-1))
                else:
                    bprob = 1 - (math.e ** (a*(-1)*(d-1)))
                Pab = probgrid[i,j] * bprob
                newgrid[i, j] = Pab / Pb
    return newgrid

def response_2_mice(probgrid_m1, botrow, botcol, a, beep):
    #Case: beep received, moved to [moverow, movecol], no mouse
    #Botrow, botcol: intial space where the sensor data was received
    newgrid_m1 = np.zeros((len(probgrid_m1[0]), len(probgrid_m1[0])))
    prob_beep = 0
    for i in range(len(probgrid_m1[0])):
        for j in range(len(probgrid_m1[0])):
            if probgrid_m1[i, j] != 0:
                d = abs(i - botrow) + abs(j - botcol)
                bprob = math.e ** (a*(-1)*(d-1))
                prob_beep += 2 * (probgrid_m1[i, j] * bprob)
                #prob_beep -= (math.e ** (a*(-1)*(d-1))) ** 2 + 2 * (math.e ** (a*(-1)*(d-1)) * (1 - (math.e ** (a*(-1)*(d-1))))) * ((probgrid_m1[i, j]) ** 2)
    #Probability that both bots set off a beep
    pBoth = 0
    for i in range(len(probgrid_m1[0])):
        for j in range(len(probgrid_m1[0])):
            if probgrid_m1[i, j] != 0:
                outDist = abs(i - botrow) + abs(j - botcol)
                for x in range(len(probgrid_m1[0])):
                    for y in range(len(probgrid_m1[0])):
                        if probgrid_m1[x, y] != 0:
                            inDist = abs(x - botrow) + abs(y - botcol)
                            pBoth += probgrid_m1[i, j] * math.e ** (a*(-1)*(outDist-1)) * probgrid_m1[x, y] * math.e ** (a*(-1)*(inDist-1))
    prob_beep_final = prob_beep - pBoth
    #Marginalize over mouse 2
    for i in range(len(probgrid_m1[0])):
        for j in range(len(probgrid_m1[0])):
            if probgrid_m1[i, j] != 0:
                newPij = 0
                outDist = abs(i - botrow) + abs(j - botcol)
                for x in range(len(probgrid_m1[0])):
                    for y in range(len(probgrid_m1[0])):
                        #print((outDist, inDist))
                        if probgrid_m1[x, y] != 0:
                            inDist = abs(x - botrow) + abs(y - botcol)
                            if beep == 1:
                                p_m1andm2_given_B = (1 - (1 - math.e ** (a*(-1)*(outDist-1))) * (1 - math.e ** (a*(-1)*(inDist-1)))) * probgrid_m1[i, j] * probgrid_m1[x, y] / prob_beep_final
                                newPij += p_m1andm2_given_B
                            else:
                                #Set m1 probs to prob m1 and m2 given no beep
                                p_m1andm2_given_NB = (1 - math.e ** (a*(-1)*(outDist-1))) * (1 - math.e ** (a*(-1)*(inDist-1))) * probgrid_m1[i, j] * probgrid_m1[x, y] / (1 - prob_beep_final)
                                newPij += p_m1andm2_given_NB

                    newgrid_m1[i, j] = newPij

    return newgrid_m1

#Update the probgrid when a bot moves to a cell and the bot isn't there
#Updates each P(Ai) to P(Ai|~M) = P(Ai ^ ~M) / P(~M) = P(Ai) / (1 - P(M))
def move(probgrid, moverow, movecol):
    #Probability that the mouse was in the cell we moved to
    moveprob = probgrid[moverow, movecol]
    newgrid = np.zeros((len(probgrid[0]), len(probgrid[0])))
    for i in range(len(probgrid[0])):
        for j in range(len(probgrid[0])):
            if probgrid[i, j] != 0:
                prevprob = probgrid[i, j]
                newgrid[i, j] = prevprob / (1 - moveprob)
    newgrid[moverow, movecol] = 0
    ##print("sum: ", np.sum(newgrid))
    return newgrid

def mouse_availability(mouserow, mousecol, ship):
    available_mouse_cells = [(mouserow, mousecol)]
    if mouserow > 0:
        if ship[mouserow-1, mousecol] == 0:
            available_mouse_cells.append((mouserow-1, mousecol))
    #check down
    if mouserow < len(ship[0])-1:
        if ship[mouserow+1, mousecol] == 0:
            available_mouse_cells.append((mouserow+1, mousecol))
    #check left
    if mousecol > 0:
        if ship[mouserow, mousecol-1] == 0:
            available_mouse_cells.append((mouserow, mousecol-1))
    #check right
    if mousecol < len(ship[0])-1:
        if ship[mouserow, mousecol+1] == 0:
            available_mouse_cells.append((mouserow, mousecol+1))
    return available_mouse_cells

def mousemove(probgrid, ship):
    newprobs = probgrid.copy()
    d = len(probgrid[0])
    #Build available_moves matrix
    available_moves = np.zeros_like(probgrid)
    for i in range(len(probgrid[0])):
        for j in range(len(probgrid[0])):
            if ship[i, j] != 1:
                k = check_adj(i, j, ship, len(ship[0]))
                available_moves[i, j] = k + 1
    for i in range(len(probgrid[0])):
        for j in range(len(probgrid[0])):
            #Obtain valid neighbors of ship[i, j]
            if ship[i, j] != 1  and ship[i, j] != 2:
                neighbors = []
                if i > 0:
                    if ship[i-1, j] != 1:
                        neighbors.append((i-1, j))
                #check down
                if i < d-1:
                    if ship[i+1, j] != 1:
                        neighbors.append((i+1, j))
                #check left
                if j > 0:
                    if ship[i, j-1] != 1:
                        neighbors.append((i, j-1))
                #check right
                if j < d-1:
                    if ship[i, j+1] != 1:
                        neighbors.append((i, j+1))
                for n in neighbors:
                    #For each neighbor n of [i, j], add the probability that the mouse was in n and moved to [i, j] P(mouse in n) * P(mouse moves to (i, j))
                    #and subtract the probability that the mouse was in [i, j] and moved to n P(mouse in (i, j)) * P(mouse moves to n)
                    if available_moves[n[0], n[1]] > 0:
                        newprobs[i, j] += probgrid[n[0], n[1]] / available_moves[n[0], n[1]]
                    if available_moves[i, j] > 0:
                        newprobs[i, j] -= probgrid[i, j] / available_moves[i, j]
    return newprobs

x_trainStationary = torch.zeros(1, 3, 40, 40)
y_trainStationary = torch.zeros(1, 1)
x_trainMoving = torch.zeros(1, 3, 40, 40)
y_trainMoving = torch.zeros(1, 1)

def bot3(dim, a, mouse_behavior, num_mice, const_res):
    #3 2x5x5 torches

    ship = const_res[0]
    botrow = const_res[1]
    botcol = const_res[2]
    mouserow = const_res[3]
    mousecol = const_res[4]
    mfound = 0
    if num_mice == 2:
        for i in range(len(ship[0])):
            for j in range(len(ship[0])):
                if ship[i, j] == 3 and mfound == 0:
                    mouserow = i
                    mousecol = j
                    mfound += 1
                if ship[i, j] == 3 and mfound == 1:
                    mouserow2 = i
                    mousecol2 = j
    probgrid = probbuild(ship)
    #print(ship)
    #print(probgrid)
    found = 0
    round = 1
    t = 0
    pathbreaks = 0
    micefound = 0
    x = torch.zeros(1, 3, dim, dim)
    y = torch.zeros(1, 1)
    while(found == 0):
        yNew = 4
        npNew = np.zeros((3, 40, 40))
        ship[mouserow, mousecol] = 0
        ship[botrow, botcol] = 0
        npNew[0] = ship
        npNew[1] = probgrid
        botLoc = np.zeros((40, 40))
        botLoc[botrow, botcol] = 1
        npNew[2] = botLoc
        xNew = torch.from_numpy(npNew)
        x[x.shape[0] - 1] = xNew
        x.resize_(x.shape[0] + 1, 3, 40, 40)
        y[y.shape[0] - 1] = yNew
        y.resize_(y.shape[0] + 1, 1)
        ship[mouserow, mousecol] = 3
        ship[botrow, botcol] = 2
        dist = abs(mouserow - botrow) + abs(mousecol - botcol)
        beepprob = math.e ** (a*(-1)*(dist-1))
        roll = random.random()
        roll2 = 1
        beepprob2 = 0
        if num_mice == 2:
            dist2 = abs(mouserow - botrow) + abs(mousecol - botcol)
            beepprob2 = math.e ** (a*(-1)*(dist2-1))
            roll2 = random.random()
        t += 1
        if roll < beepprob:
            #Beep received
            #print("Beep received")
            if (num_mice == 1) or (num_mice == 2 and micefound == 1):
                probgrid = response(probgrid, a, botrow, botcol, 1)
            else:
                probgrid = response_2_mice(probgrid, botrow, botcol, a, 1)
        else:
            #No beep received
            #print("No beep received")
            if (num_mice == 1) or (num_mice == 2 and micefound == 1):
                probgrid = response(probgrid, a, botrow, botcol, 0)
            else:
                probgrid = response_2_mice(probgrid, botrow, botcol, a, 0)
        #print("Sense")
        #print(probgrid)
        if mouse_behavior == 1:
            if mouserow != -1 and mousecol != -1:
                #Stochastic mouse: allow the mouse to move or stay in its cell with uniform probability
                available_mouse_cells = mouse_availability(mouserow, mousecol, ship)
                #print("available mouse cellse: ", available_mouse_cells)
                mouse_move_cell = random.choice(available_mouse_cells)
                ship[mouserow, mousecol] = 0
                ship[mouse_move_cell[0], mouse_move_cell[1]] = 3
                mouserow = mouse_move_cell[0]
                mousecol = mouse_move_cell[1]
                #Now that the mouse has moved, the bot must reevaluate the probgrid considering that the mouse could have moved
                probgrid = mousemove(probgrid, ship)
            if num_mice == 2:
                if mouserow2 != -1 and mousecol2 != -1:
                    available_mouse_cells = mouse_availability(mouserow2, mousecol2, ship)
                    #print("available mouse cellse: ", available_mouse_cells)
                    mouse_move_cell = random.choice(available_mouse_cells)
                    ship[mouserow2, mousecol2] = 0
                    ship[mouse_move_cell[0], mouse_move_cell[1]] = 3
                    mouserow2 = mouse_move_cell[0]
                    mousecol2 = mouse_move_cell[1]
                    probgrid = mousemove(probgrid, ship)

        #Probgrid updates: Identify cell with highest probability, break ties with proximity to bot, further break ties with randomness
        ##print(np.unravel_index(np.argmax(probgrid, axis=None), probgrid.shape))
        ##print(np.max(probgrid))
        maxes = []
        for i in range(len(probgrid[0])):
            for j in range(len(probgrid[0])):
                if probgrid[i, j] == np.max(probgrid):
                    maxes.append((i, j))
        mindist = np.inf
        for m in maxes:
            if abs(m[0] - botrow) + abs(m[1] - botcol) < mindist:
                mindist = abs(m[0] - botrow) + abs(m[1] - botcol)
        for m in maxes:
            if abs(m[0] - botrow) + abs(m[1] - botcol) > mindist:
                maxes.remove(m)
        dest = random.choice(maxes)
        #Plot path to that cell
        path = bfs(ship, botrow, botcol, dim, dest[0], dest[1])
        round += 1
        # y: [up (0), down (1), left (2), right (3), sense(4)]
        #Loop: Proceed to next cell in path, check for mouse, if there is no mouse update the probabilities by calling move
        for i in range(1, len(path)):
            #Check for mouse
            #yNew = 4
            if path[i][0] == botrow - 1:
                #moving up
                yNew = 0
            if path[i][0] == botrow + 1:
                #moving down
                yNew = 1
            if path[i][1] == botcol - 1:
                #left
                yNew = 2
            if path[i][1] == botcol + 1:
                #right
                yNew = 3
            npNew = np.zeros((3, 40, 40))
            ship[mouserow, mousecol] = 0
            ship[botrow, botcol] = 0
            npNew[0] = ship
            npNew[1] = probgrid
            botLoc = np.zeros((40, 40))
            botLoc[botrow, botcol] = 1
            npNew[2] = botLoc
            xNew = torch.from_numpy(npNew)
            x[x.shape[0] - 1] = xNew
            x.resize_(x.shape[0] + 1, 3, 40, 40)
            y[y.shape[0] - 1] = yNew
            y.resize_(y.shape[0] + 1, 1)
            ship[mouserow, mousecol] = 3
            ship[botrow, botcol] = 2
            if ship[path[i][0], path[i][1]] == 3:
                t += 1
                found += 1
                ship[path[i][0], path[i][1]] = 2
                ship[path[i-1][0], path[i-1][1]] = 0
                botrow = path[i][0]
                botcol = path[i][1]
                #print(ship)
                #print(probgrid)
                #print("Mouse found at time ", t)
                break
            else:
                ship[path[i][0], path[i][1]] = 2
                ship[path[i-1][0], path[i-1][1]] = 0
                t += 1
                botrow = path[i][0]
                botcol = path[i][1]
                probgrid = move(probgrid, botrow, botcol)
                #print(ship)
                #print(probgrid)
                flat = np.ndarray.flatten(probgrid)
                options = [x for x in flat if x > 0]
                mean = np.mean(options)
                varnum = 0
                for i in options:
                    varnum += (i - mean) ** 2
                verdenom = len(options) - 1
                variance = varnum / verdenom
                if variance < 0.0001 and pathbreaks < 50:
                    pathbreaks += 1
                    break
    x.resize_(x.shape[0] - 1, 3, 40, 40)
    y.resize_(y.shape[0] - 1, 1)
    return t, x, y

constShip = np.array([[0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0,
  1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,],
 [0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0,
  0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0,],
 [0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1,
  1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1,],
 [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0,
  0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1,],
 [0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1,
  0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0,],
 [0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0,
  0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0,],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
  0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0,],
 [1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1,
  1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1,],
 [0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0,
  0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0,],
 [1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1,
  1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0,],
 [0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0,
  0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0,],
 [0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1,
  0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1,],
 [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1,
  0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0,],
 [0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0,
  1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1,],
 [0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1,
  0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0,],
 [0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1,
  0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1,],
 [0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0,
  0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,],
 [0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
  0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1,],
 [0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1,
  0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0,],
 [0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0,
  0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1,],
 [0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0,
  1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,],
 [0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1,
  0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0,],
 [1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1,
  1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0,],
 [0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0,
  0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0,],
 [1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1,
  0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1,],
 [0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1,
  0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,],
 [0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0,
  1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0,],
 [1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0,
  0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,],
 [0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0,
  0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0,],
 [0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1,
  1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0,],
 [0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0,
  0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0,],
 [1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0,
  1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0,],
 [1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0,
  1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0,],
 [0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1,
  0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0,],
 [0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0,
  1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0,],
 [1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0,
  0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1,],
 [0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0,
  0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0,],
 [0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1,
  1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1,],
 [0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0,
  0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0,],
 [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0,
  1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,]])

c = 0
while c < 3000:
    #Start the mouse in a different cell on the ship each time
    botrow = randrange(40)
    botcol = randrange(40)
    bot_mouse_ship = constShip.copy()
    #Bot = 2
    while bot_mouse_ship[botrow, botcol] != 0:
        botrow = randrange(40)
        botcol = randrange(40)
    bot_mouse_ship[botrow, botcol] = 2
    mouserow = randrange(40)
    mousecol = randrange(40)
    #Mouse = 3
    while bot_mouse_ship[mouserow, mousecol] != 0:
        mouserow = randrange(40)
        mousecol = randrange(40)
    bot_mouse_ship[mouserow, mousecol] = 3
    const_res = [bot_mouse_ship, botrow, botcol, mouserow, mousecol]
    #Stationary:
    res = bot3(40, 0.5, 0, 1, const_res)
    #Moving:
    #res = bot3(40, 0.5, 1, 1, const_res)
    x = res[1]
    y = res[2]
    c += x.shape[0]
    #x_trainStationary = torch.cat((x_trainStationary, x))
    #y_trainStationary = torch.cat((y_trainStationary, y))
    #x_trainMoving = torch.cat((x_trainMoving, x))
    #y_trainMoving = torch.cat((y_trainMoving, y))

torch.save(x_trainStationary, 'x_train_Stationary.pt')
torch.save(y_trainStationary, 'y_train_Stationary.pt')
#torch.save(x_trainMoving, 'x_testMovingsensing.pt')
#torch.save(y_trainMoving, 'y_testMovingsensing.pt')

In [7]:
x = torch.load('x_train_Stationary.pt')
print(x.shape)

torch.Size([1, 3, 40, 40])
